In [1]:
#Joshua Bernstein
# Simulation of Differential Fault Attack on DES

import math
import numpy as np
import scipy
import random
import csv
import random
import string
from itertools import chain
import numpy as numpy
from pydes import des

# inverse final permutation table
IP =   [40, 8, 48, 16, 56, 24, 64, 32,
        39, 7, 47, 15, 55, 23, 63, 31,
        38, 6, 46, 14, 54, 22, 62, 30,
        37, 5, 45, 13, 53, 21, 61, 29,
        36, 4, 44, 12, 52, 20, 60, 28,
        35, 3, 43, 11, 51, 19, 59, 27,
        34, 2, 42, 10, 50, 18, 58, 26,
        33, 1, 41, 9, 49, 17, 57, 25]

# S boxes
S = [

    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],
     ],

    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],
     ],

    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],
     ],

    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],
     ],

    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],
     ],

    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],
     ],

    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],
     ],

    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
     ]
]

# converts binary list to hex
def binary_to_hex(binary_list):
    # Convert binary list to string and pad with leading zeros if necessary
    binary_string = ''.join(map(str, binary_list)).zfill(4 * ((len(binary_list) + 3) // 4))

    # Convert binary string to hex string
    hex_string = hex(int(binary_string, 2))[2:]

    return hex_string

# converts binary list to integer
def binary_to_int(in_bin):
    # Convert the binary list to a string
    binary_string = ''.join([str(bit) for bit in in_bin])

    # Convert the binary string to an integer
    decimal = int(binary_string, 2)

    return decimal

# convert integer to binary list
def int_to_binary(in_dec):
    # Convert the decimal number to a binary string with 4 digits
    binary = format(in_dec, '04b')

    # Convert the binary string to a list of integers
    binary_list = [int(bit) for bit in binary]

    return binary_list


# function to reverse final permutation to get L16 and R16
def undo_final_permutation(ciphertext, final_permutation_table):
    # Convert the ciphertext from binary string to a list of bits
    ciphertext_bits = [int(bit) for bit in ciphertext]

    # Apply the inverse final permutation to the ciphertext
    plaintext_bits = [0] * len(ciphertext_bits)
    for i, position in enumerate(final_permutation_table):
        plaintext_bits[i] = ciphertext_bits[position - 1]

    # Convert the plaintext from a list of bits to a binary string
    plaintext = ''.join([str(bit) for bit in plaintext_bits])

    return plaintext_bits

# function to separate into left and right halves
def separate(binary_string):
    right = binary_string[:32]
    left = binary_string[32:]
    return left, right

# function to perform bitwise or
def bitwise_xor(in1, in2):
    result = []

    if len(in1) != len(in2):
        print("xor input length error")
        print("in 1 has length", len(in1))
        print("in 2 has length", len(in2))
        raise ValueError("")

    if (in1[0] == 9 ) or (in2[0] == 9):
        for i in range(len(in1)):
            result.append(9)

    # Compute the bitwise XOR of the two lists of bits
    else:
        for i in range(len(in1)):
            result.append(in1[i] ^ in2[i])
        return result

# array flatten funciton
def flatten(lst):
    return [item for sublist in lst for item in sublist]

# function to generate random plaintext
def generate_plaintext():
    return ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits + string.punctuation, k=8))


# expansion function
def expand(input):
    expanded = []
    E = [
        32,  1,  2,  3,  4,  5,
        4,  5,  6,  7,  8,  9,
        8,  9, 10, 11, 12, 13,
        12, 13, 14, 15, 16, 17,
        16, 17, 18, 19, 20, 21,
        20, 21, 22, 23, 24, 25,
        24, 25, 26, 27, 28, 29,
        28, 29, 30, 31, 32,  1]
    for i in E:
        expanded.append(input[i-1])
    return expanded



# create array of all possible 6 bit values for S-boxes
all_possibilites = []


for i in range(2 ** 6):
    binary_str = bin(i)[2:].zfill(6)
    all_possibilites.append([int(bit) for bit in binary_str])



######################################### begin fault attack ########################################
# key
key = "secret_k"

# flag to check if it is the first loop round or not
init_flag = 1

# flag to check if there is only one possible key for each s box
done_flag = 0

# bit will be set to 1 if there is only 1 possibility for that s box
done_checks = [0,0,0,0,0,0,0,0]

# array to hold key_16
key_16 = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

# create instance of des
d = des()



########## start loop here ##########
while done_flag != 1:
    # generate random fault bit and random plaintext
    fault_bit = random.randint(0, 31)
    text= generate_plaintext()

    # correct encryption
    ciphered, bin = d.encrypt(key,text)

    # encryption with fault injected in round 15
    cipheredFault, binFault = d.encrypt(key,text, fault = True, roundNum=15, bitNum=fault_bit)

    # undo final permutation on correct and faulty ciphertext
    output = undo_final_permutation(bin, IP)
    outputFault = undo_final_permutation(binFault,IP)

    # separate correct and faulty outputs of final round into left and right halves
    L16, R16 = separate(output)
    L16_f, R16_f = separate(outputFault)

    # xor correct and faulty right halves of round 16 output
    xor_R = bitwise_xor(R16,R16_f)

    # split into 4 bit chunks for later comparison
    xor_R_4 = [xor_R[i:i+4] for i in range(0, len(xor_R), 4)]

    # expand L16 and L16_f (which is the same as R15)
    L16_exp = expand(L16)
    L16_f_exp = expand(L16_f)

    # arrays to hold tested keys
    xor_L = []
    xor_LF = []
    xor_L_temp = []
    xor_LF_temp = []


    # perform (L16 xor key) and (L16^ xor key)
    for j in range(0, 48, 6):
        for i in range(2**6):



            # calculate xor of correct expanded left half with all potential keys
            xor_L_temp = bitwise_xor(L16_exp[j:j+6],all_possibilites[i][:])
            xor_L.append(xor_L_temp)

            # calculate xor of faulty expanded left half with al potential keys
            xor_LF_temp = bitwise_xor(L16_f_exp[j:j+6], all_possibilites[i][:])
            xor_LF.append(xor_LF_temp)





    # separate into 8 columns containing xor results with each possible 6 bit key chunk
    # each column corresponds to a set of 6 bits from the 48 bit key, each row represents each possible 6 bit key chunk
    xor_L_sorted = [[] for _ in range(64)]
    for i, row in enumerate(xor_L):
        xor_L_sorted[i // 8].append(row)

    # do the same for the faulty xor results
    xor_LF_sorted = [[] for _ in range(64)]
    for i, row in enumerate(xor_LF):
        xor_LF_sorted[i // 8].append(row)



    ######################################### s box process ########################################

    # holds s box row and col address bits (_b = correct, _bf = faulty)
    row_addr_b = []
    row_addr_bf = []
    col_addr_b = []
    col_addr_bf = []

    # sbox outputs for correct and faulty cipher

    sbox_out_temp = []
    sbox_out_f_temp = []

    # loop over rows (key vals)
    for j in range(len(xor_L_sorted[0])):
        # loop over columns (key chunks)
        for i in range(len(xor_L_sorted)):

            # get current 6 bit string
            temp = xor_L_sorted[i][j]
            temp_f = xor_LF_sorted[i][j]

            # convert into s box address
            row_addr_b = binary_to_int([temp[0], temp[5]])
            col_addr_b = binary_to_int(temp[1:5])

            #do the same for faulty
            row_addr_bf = binary_to_int([temp_f[0], temp_f[5]])
            col_addr_bf = binary_to_int(temp_f[1:5])

            # get s box output bits
            sbox_out_temp.append(int_to_binary(S[j][row_addr_b][col_addr_b]))

            # do the same for faulty
            sbox_out_f_temp.append(int_to_binary(S[j][row_addr_bf][col_addr_bf]))


    sbox_out = [[] for _ in range(64)]
    for i, row in enumerate(sbox_out_temp):
            sbox_out[i // 8].append(row)

    sbox_out_f = [[] for _ in range(64)]
    for i, row in enumerate(sbox_out_f_temp):
            sbox_out_f[i // 8].append(row)




    ######################################### finish equation  ########################################
    L_final = [[] for i in range(len(sbox_out))]
    count = [0,0,0,0,0,0,0,0]


    for j in range(len(sbox_out[0])):
        current_possibilites = []
        for i in range(len(sbox_out)):

            # append empty line
            L_final[i].append([])

            # xor each correct sbox output with corresponding faulty sbox output
            temp1 = sbox_out[i][j]
            temp2 = sbox_out_f[i][j]

            # get final left half of equation
            L_final[i][j] = bitwise_xor(temp1,temp2)

            # add all current possibilites for current s box to list
            if L_final[i][j] == xor_R_4[j]:
                 current_possibilites.append(all_possibilites[i])
                 count[j] += 1

        # if there is only 1 possibility for current s box, set key16 for that chunk
        if count[j] == 1:
            key_16[j] = current_possibilites[0]
            done_checks[j] = 1




    # if all s boxes have only 1 key possibility set flag and exit loop
    if done_checks == [1,1,1,1,1,1,1,1]:
        done_flag = 1

# flatten round key 16
key_16 = flatten(key_16)
print(key_16)

[1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
